In [2]:
categories = ['alt.atheism', 'soc.religion.christian',
...               'comp.graphics', 'sci.med', 'sci.space']

In [3]:
from sklearn.datasets import fetch_20newsgroups

In [4]:
twenty_train = fetch_20newsgroups(subset='train',
...     categories=categories, shuffle=True, random_state=42)


In [19]:
twenty_train.target_names


['alt.atheism',
 'comp.graphics',
 'sci.med',
 'sci.space',
 'soc.religion.christian']

In [20]:
len(twenty_train.data)
len(twenty_train.filenames)

2850

In [21]:
print(twenty_train.data[0])

From: ddeciacco@cix.compulink.co.uk (David Deciacco)
Subject: Re: Another CVIEW question (wa
Reply-To: ddeciacco@cix.compulink.co.uk
Lines: 5


In-Reply-To: <20APR199312262902@rigel.tamu.edu> lmp8913@rigel.tamu.edu (PRESTON, LISA M)

I have a trident card and fullview works real gif jpg try it#
dave



In [22]:
print("\n".join(twenty_train.data[0].split("\n")[:3]))

From: ddeciacco@cix.compulink.co.uk (David Deciacco)
Subject: Re: Another CVIEW question (wa
Reply-To: ddeciacco@cix.compulink.co.uk


In [24]:
print(twenty_train.target_names[twenty_train.target[0]])

comp.graphics


Supervised learning algorithms will require a category label for each document in the training set. In this case the category is the name of the newsgroup which also happens to be the name of the folder holding the individual documents.

In [179]:
twenty_train.target[:10]

array([1, 2, 2, 2, 1, 3, 0, 0, 4, 3])

In [180]:
for t in twenty_train.target[:10]:
...     print(twenty_train.target_names[t])

comp.graphics
sci.med
sci.med
sci.med
comp.graphics
sci.space
alt.atheism
alt.atheism
soc.religion.christian
sci.space


You can notice that the samples have been shuffled randomly (with a fixed RNG seed): this is useful if you select only the first samples to quickly train a model and get a first idea of the results before re-training on the complete dataset later.

# Tokenizing text with scikit-learn

Text preprocessing, tokenizing and filtering of stopwords are included in a high level component that is able to build a dictionary of features and transform documents to feature vectors:

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(2850, 42370)

**CountVectorizer** supports counts of N-grams of words or consecutive characters. Once fitted, the vectorizer has built a dictionary of feature indices:

In [29]:
count_vect.vocabulary_.get(u'algorithm')

6089

The index value of a word in the vocabulary is linked to its frequency in the whole training corpus.

# From occurrences to frequencies

Occurrence count is a good start but there is an issue: longer documents will have higher average count values than shorter documents, even though they might talk about the same topics.

To avoid these potential discrepancies it suffices to divide the number of occurrences of each word in a document by the total number of words in the document: these new features are called tf for Term Frequencies.

Another refinement on top of tf is to downscale weights for words that occur in many documents in the corpus and are therefore less informative than those that occur only in a smaller portion of the corpus.

This downscaling is called tf–idf for “Term Frequency times Inverse Document Frequency”.

Both tf and tf–idf can be computed as follows:

In [30]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(2850, 42370)

In the above example-code, we firstly use the fit(..) method to fit our estimator to the data and secondly the transform(..) method to transform our count-matrix to a tf-idf representation. These two steps can be combined to achieve the same end result faster by skipping redundant processing. This is done through using the fit_transform(..) method as shown below, and as mentioned in the note in the previous section:

In [31]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2850, 42370)

# Training a classifier

Now that we have our features, we can train a classifier to try to predict the category of a post. Let’s start with a naïve Bayes classifier, which provides a nice baseline for this task. scikit-learn includes several variants of this classifier; the one most suitable for word counts is the multinomial variant:

In [36]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)


To try to predict the outcome on a new document we need to extract the features using almost the same feature extracting chain as before. The difference is that we call transform instead of fit_transform on the transformers, since they have already been fit to the training set:

In [42]:
docs_new = ['God is love', 'OpenGL on the GPU is fast','i dont belive in atheism']

In [43]:
X_new_counts = count_vect.transform(docs_new)

In [44]:
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

In [45]:
predicted = clf.predict(X_new_tfidf)

In [46]:
for doc, category in zip(docs_new, predicted):
...     print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics
'i dont belive in atheism' => alt.atheism


# Building a pipeline

In order to make the vectorizer => transformer => classifier easier to work with, scikit-learn provides a Pipeline class that behaves like a compound classifier:

In [48]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
])

In [49]:
text_clf.fit(twenty_train.data, twenty_train.target) 

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

# Evaluation of the performance on the test set

In [193]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test',
    categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target) 

0.8422995780590717

I.e., we achieved 83.4% accuracy. Let’s see if we can do better with a linear support vector machine (SVM), which is widely regarded as one of the best text classification algorithms (although it’s also a bit slower than naïve Bayes). We can change the learner by just plugging a different classifier object into our pipeline:

In [194]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, random_state=42,
                                           max_iter=5, tol=None)),
])
text_clf.fit(twenty_train.data, twenty_train.target)  

predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)  

0.9092827004219409

scikit-learn further provides utilities for more detailed performance analysis of the results:

In [195]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted,
    target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.95      0.79      0.86       319
         comp.graphics       0.87      0.96      0.91       389
               sci.med       0.94      0.88      0.91       396
             sci.space       0.95      0.94      0.94       394
soc.religion.christian       0.87      0.96      0.91       398

           avg / total       0.91      0.91      0.91      1896



In [31]:
metrics.confusion_matrix(twenty_test.target, predicted)

array([[258,  11,  15,  35],
       [  4, 379,   3,   3],
       [  5,  33, 355,   3],
       [  5,  10,   4, 379]])